In [1]:
########################################
########################################
########################################
########################################
##Design by LCY

!unzip -q data/data80164/train_and_label.zip
!unzip -q data/data80164/img_test.zip

!pip install paddlex -i https://mirror.baidu.com/pypi/simple
!pip install imgaug -i https://mirror.baidu.com/pypi/simple
import matplotlib
import os
import paddlex as pdx
import numpy as np
from paddlex.seg import transforms
import imgaug.augmenters as iaa
#import paddle
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
#paddle.set_device('gpu:0')


# 定义训练和验证时的transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize(target_size=300),
    transforms.RandomPaddingCrop(crop_size=256),
    transforms.RandomBlur(prob=0.02),
    transforms.RandomRotate(rotate_range=15),
    transforms.RandomDistort(brightness_range=0.1, contrast_range=0.1, saturation_range=0.1, hue_prob=0),
    transforms.Normalize()
])
eval_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.Normalize()
])



datas = []
image_base = 'img_train'   # 训练集原图路径
annos_base = 'lab_train'   # 训练集标签路径

# 读取原图文件名
ids_ = [v.split('.')[0] for v in os.listdir(image_base)]

# 将训练集的图像集和标签路径写入datas中
for id_ in ids_:
    img_pt0 = os.path.join(image_base, '{}.jpg'.format(id_))
    img_pt1 = os.path.join(annos_base, '{}.png'.format(id_))
    datas.append((img_pt0.replace('/home/aistudio', ''), img_pt1.replace('/home/aistudio', '')))
    if os.path.exists(img_pt0) and os.path.exists(img_pt1):
        pass
    else:
        raise "path invalid!"

# 打印datas的长度和具体存储例子
#print('total:', len(datas))
#print(datas[0][0])
#print(datas[0][1])
#print(datas[10][:])


labels = ['建筑', '耕地', '林地',  '其他']

# 将labels写入标签文件
with open('labels.txt', 'w') as f:
    for v in labels:
        f.write(v+'\n')

# 随机打乱datas
np.random.seed(5)
np.random.shuffle(datas)

# 验证集与训练集的划分，0.05表示5%为训练集，95%为训练集
split_num = int(0.05*len(datas))

# 划分训练集和验证集
train_data = datas[:-split_num]
valid_data = datas[-split_num:]

# 写入训练集list
with open('train_list.txt', 'w') as f:
    for img, lbl in train_data:
        f.write(img + ' ' + lbl + '\n')

# 写入验证集list
with open('valid_list.txt', 'w') as f:
    for img, lbl in valid_data:
        f.write(img + ' ' + lbl + '\n')

# 打印训练集和测试集大小
#print('train:', len(train_data))
#print('valid:', len(valid_data))

data_dir = './'

# 定义训练和验证数据集
train_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,                # 数据集路径
    file_list='train_list.txt',       # 训练集图片文件list路径
    label_list='labels.txt',          # 训练集标签文件list路径
    transforms=train_transforms,      # train_transforms
    shuffle=True)                     # 数据集是否打乱
    
eval_dataset = pdx.datasets.SegDataset(
    data_dir=data_dir,                # 数据集路径
    file_list='valid_list.txt',       # 验证集图片文件list路径
    label_list='labels.txt',          # 验证集标签文件list路径
    transforms=eval_transforms)       # eval_transforms

    # 分割类别数
num_classes = len(train_dataset.labels)

# 构建DeepLabv3p分割器
model = pdx.seg.DeepLabv3p(
    num_classes=num_classes,  backbone='Xception65', use_bce_loss=False
)

# 模型训练
model.train(
    num_epochs=10,                 # 训练迭代轮数
    train_dataset=train_dataset,  # 训练集读取
    train_batch_size=32,           # 训练时批处理图片数
    eval_dataset=eval_dataset,    # 验证集读取
    learning_rate=0.0016,         # 学习率
    save_interval_epochs=1,       # 保存模型间隔轮次  
    save_dir='work/output/deeplab',    # 模型保存路径
    log_interval_steps=200,       # 日志打印间隔
    pretrain_weights='work/output/deeplab/best_model')  #加载预训练模型

    # 加载模型
model = pdx.load_model('work/output/deeplab/best_model')

# 模型评估
model.evaluate(eval_dataset, batch_size=1, epoch_id=None, return_details=False)



from tqdm import tqdm
import cv2
import os

test_base = 'img_testA/'    # 测试集路径
out_base = 'result/'        # 预测结果保存路径

# 是否存在结果保存路径，如不存在，则创建该路径
if not os.path.exists(out_base):
    os.makedirs(out_base)

# 模型预测并保存预测图片
for im in tqdm(os.listdir(test_base)):
    if not im.endswith('.jpg'):
        continue
    pt = test_base + im
    result = model.predict(pt)
    cv2.imwrite(out_base+im.replace('jpg', 'png'), result['label_map'])

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


2021-05-21 13:38:49 [INFO]	63320 samples in file train_list.txt
2021-05-21 13:38:49 [INFO]	3332 samples in file valid_list.txt


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/xception.py:316
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:687: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release

2021-05-21 13:38:58 [INFO]	Load pretrain weights from work/output/deeplab/best_model.
2021-05-21 13:39:00 [INFO]	There are 732 varaibles in work/output/deeplab/best_model are loaded.
2021-05-21 13:40:25 [INFO]	[TRAIN] Epoch=1/10, Step=200/1978, loss=0.541217, lr=0.001586, time_each_step=0.39s, eta=2:14:47
2021-05-21 13:41:43 [INFO]	[TRAIN] Epoch=1/10, Step=400/1978, loss=0.491579, lr=0.001571, time_each_step=0.39s, eta=2:13:24
2021-05-21 13:43:02 [INFO]	[TRAIN] Epoch=1/10, Step=600/1978, loss=0.353769, lr=0.001556, time_each_step=0.39s, eta=2:12:0
2021-05-21 13:44:21 [INFO]	[TRAIN] Epoch=1/10, Step=800/1978, loss=0.245873, lr=0.001542, time_each_step=0.39s, eta=2:11:4
2021-05-21 13:45:39 [INFO]	[TRAIN] Epoch=1/10, Step=1000/1978, loss=0.356098, lr=0.001527, time_each_step=0.39s, eta=2:9:33
2021-05-21 13:46:58 [INFO]	[TRAIN] Epoch=1/10, Step=1200/1978, loss=0.362123, lr=0.001512, time_each_step=0.4s, eta=2:9:29
2021-05-21 13:48:16 [INFO]	[TRAIN] Epoch=1/10, Step=1400/1978, loss=0.533718

100%|██████████| 105/105 [00:22<00:00,  4.60it/s]


...

2021-05-21 15:41:01 [INFO]	[EVAL] Finished, Epoch=9, miou=0.68556, category_iou=[0.69549976 0.80906998 0.67056025 0.56711032], oacc=0.824843, category_acc=[0.83176894 0.89217583 0.79209496 0.72316845], kappa=0.76118, category_F1-score=[0.82040679 0.89445957 0.80279685 0.72376567] .
2021-05-21 15:41:05 [INFO]	Model saved in work/output/deeplab/best_model.
2021-05-21 15:41:09 [INFO]	Model saved in work/output/deeplab/epoch_9.
2021-05-21 15:41:09 [INFO]	Current evaluated best model in eval_dataset is epoch_9, miou=0.6855600800396153
2021-05-21 15:42:33 [INFO]	[TRAIN] Epoch=10/10, Step=198/1978, loss=0.260095, lr=0.000183, time_each_step=0.39s, eta=0:12:10
2021-05-21 15:43:51 [INFO]	[TRAIN] Epoch=10/10, Step=398/1978, loss=0.166212, lr=0.000165, time_each_step=0.39s, eta=0:10:50
2021-05-21 15:45:10 [INFO]	[TRAIN] Epoch=10/10, Step=598/1978, loss=0.281432, lr=0.000146, time_each_step=0.39s, eta=0:9:33
2021-05-21 15:46:29 [INFO]	[TRAIN] Epoch=10/10, Step=798/1978, loss=0.149483, lr=0.000127,

100%|██████████| 105/105 [00:23<00:00,  4.52it/s]


2021-05-21 15:54:35 [INFO]	[EVAL] Finished, Epoch=10, miou=0.689498, category_iou=[0.70179479 0.8098497  0.67362632 0.57272045], oacc=0.827298, category_acc=[0.84237554 0.8930642  0.79202232 0.72172886], kappa=0.764659, category_F1-score=[0.82477017 0.89493586 0.80499011 0.72831818] .
2021-05-21 15:54:40 [INFO]	Model saved in work/output/deeplab/best_model.
2021-05-21 15:54:44 [INFO]	Model saved in work/output/deeplab/epoch_10.
2021-05-21 15:54:44 [INFO]	Current evaluated best model in eval_dataset is epoch_10, miou=0.6894978151027245
2021-05-21 15:54:47 [INFO]	Model[DeepLabv3p] loaded.
2021-05-21 15:54:47 [INFO]	Start to evaluating(total_samples=3332, total_steps=3332)...


100%|██████████| 3332/3332 [00:53<00:00, 62.78it/s]


OrderedDict([('miou', 0.6894978151027245),
             ('category_iou',
              array([0.70179479, 0.8098497 , 0.67362632, 0.57272045])),
             ('oacc', 0.8272980828757897),
             ('category_acc',
              array([0.84237554, 0.8930642 , 0.79202232, 0.72172886])),
             ('kappa', 0.7646587134915591),
             ('category_F1-score',
              array([0.82477017, 0.89493586, 0.80499011, 0.72831818]))])